### Домашнее задание. Коллоборативная фильтрация

- используйте данные MovieLens 1M
- можно использовать любые модели из пакета
- получите RMSE на тестовом сете 0.87 и ниже

In [2]:
pip install scikit-surprise

  Stored in directory: C:\Users\38E2~1\AppData\Local\pip\Cache\wheels\cc\fa\8c\16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise
Note: you may need to restart the kernel to use updated packages.


In [38]:
from surprise import KNNWithMeans, KNNBasic, SVD, BaselineOnly
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
import pandas as pd
import numpy as np
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV

комп не тянет загрузку данных. использовал маленький DF

In [2]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [3]:
data = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
data.dropna(inplace=True)

In [4]:
print(f'Количество фильмов: {data.movieId.unique().shape[0]}')
print(f'Количество пользователей: {data.userId.unique().shape[0]}')

Количество фильмов: 9724
Количество пользователей: 610


In [5]:
# определяем данные для дальнейшего обучения в формате user item rating
dataset = pd.DataFrame({
    'uid': data.userId,
    'iid': data.title,
    'rating': data.rating
})

In [42]:
dataset.shape

(100836, 3)

In [7]:
# для дальнейшей настройки алгоритма определим минимум и максимум значений в таргете
print(f'Минимальная оценка: {dataset.rating.min()}')
print(f'Максимальная оценка: {dataset.rating.max()}')

Минимальная оценка: 0.5
Максимальная оценка: 5.0


In [8]:
# формируем данные в необходимом формате для surprise
reader = Reader(rating_scale=(0.5, 5.0))
data_sur = Dataset.load_from_df(dataset, reader)

In [9]:
# делим данные на тренировочную и тестовую выборку
trainset, testset = train_test_split(data_sur, test_size=.15)

In [35]:
# выбрал алгоритмы которые посмотрю на качество
algoritms = [('KNNWithMeans', KNNWithMeans()), ('KNNBasic', KNNBasic()), ('SVD', SVD()), ('BaselineOnly', BaselineOnly())]

In [36]:
# делаю перебор моделей и смотрию их качество
for model in algoritms:
    algo = model[1]
    algo.fit(trainset)
    test_pred = algo.test(testset)
    cros_val = cross_validate(algo, data_sur, measures=['RMSE'], cv=5, verbose=True)
    RMSE_mean = cros_val['test_rmse'].mean()
    print(f'{model[0]}: {RMSE_mean:.2f}')

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8936  0.8928  0.8991  0.8927  0.9033  0.8963  0.0042  
Fit time          0.21    0.24    0.23    0.24    0.24    0.23    0.01    
Test time         3.48    3.37    3.52    3.34    3.39    3.42    0.07    
KNNWithMeans: 0.90
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...


т.о. низкую ошибку показывают алгоритмы SVD и BaselineOnly  
беру SVD для дальнейшей работы, подбора оптимальных параметров

In [39]:
# выберу параметры для SVD по которым буду делать перебор

param_grid = {'n_epochs': [5, 10, 20, 30], 'lr_all': [0.001, 0.005, 0.007, 0.01],
              'reg_all': [0.01, 0.7, 0.1, 0.5], 'random_state':[77, 777]}

# использую GridSearchCV для перебора параметров
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)

gs.fit(data_sur)

In [44]:
# вывожу оптимальные параметры алгоритма
print(gs.best_params['rmse'])

{'n_epochs': 30, 'lr_all': 0.01, 'reg_all': 0.1, 'random_state': 777}


In [47]:
# обучаю модель на оптимальных параметрах и делаю оценку
algo_best = SVD(n_epochs=30, lr_all=0.01, reg_all=0.1, random_state=777)
algo_best.fit(trainset)
test_pred = algo_best.test(testset)
cros_val = cross_validate(algo_best, data_sur, measures=['RMSE'], cv=5, verbose=True)
RMSE_mean = cros_val['test_rmse'].mean()
print(f'SVD: {RMSE_mean:.2f}')

SVD: 0.85


т.о. после подбора оптимальных параметров для алгоритма SVD получил на кросс валидации RMSE 0,85